## Import statements

In [1]:
import datetime
from scipy import stats
import time
import pickle
import pandas as pd
import gpxpy
from matplotlib import pyplot as plt
import pyproj
import georaster as gr
import numpy as np
import os

## File structure
The relations of each file to the specific flight are noted here.

**radar_data** specifies the interpreted shot count - return time data. **tracks** specifies the track gpx (or custom pickled format) name and **wpt_sync** specifies which method to use for synchronizing the RADAR and GNSS time: using waypoint/shot count or GNSS time / shot count. **waypoints** specifies the gpx waypoint file to use, if that type of synchronization is chosen. **sync_file** specifies the relation between waypoint number, or GNSS time, and shot count. **doubled** is a boolean value for if the x-values in **radar_data** are stretched to twice their length (to help the previous interpretation part).

A folder named **input** should be present in the same folder as the script. In this folder, five subfolders: **a_radar_data, b_shot_vs_wpt, b_waypoints, c_tracks** should be present, as well as **data_structure.csv**

In [2]:
structure = pd.read_csv("input/data_structure.csv", index_col=0)

structure  # Preview

,radar_data,tracks,wpt_sync,waypoints,sync_file,doubled
name,,,,,,
erik1,erik1_out_171212.txt,MARMA17C.gpx,True,Marma17C Waypoints.gpx,erik1_waypoints.csv,False
erik2,erik2_out_171212.txt,MARMA17C.gpx,True,Marma17C Waypoints.gpx,erik2_waypoints.csv,False
erik3,erik3_out_171212.txt,MARMA17C.gpx,True,Marma17C Waypoints.gpx,erik3_waypoints.csv,False
klara2,klara2_out_171212.txt,2014_radar.gpx,False,NaN,klara2_times.csv,True
anders1,anders1_out_171220.txt,anders1_track.p,False,NaN,anders1_times.csv,True


## Step A: Read RADAR data
The RADAR data are structured as shot number (relative time), return time and return type. This step reads the radar data csv table and formats it to a Pandas DataFrame.

In [3]:
def a_read_radar(in_data, flight):
    """Reads radar csv tables and formats them to a DataFrame
    :param flights: List of flights to process.
    """
    doubled = structure.loc[flight, "doubled"]
    
    values = pd.read_csv("input/a_radar_data/" + structure.loc[flight, "radar_data"])
    
    values.sort_values("x", inplace=True)
    values.y = values.y * -1  # The y-values should be positive

    if doubled:  # If x-axis was doubled during interpretation
        values.x = values.x * 0.5

    return values[["x", "y", "typ"]]

**Data preview**

In [4]:
preview_flight = "anders1"
a_preview_data = a_read_radar(in_data=None, flight=preview_flight)
a_preview_data.head()

,x,y,typ
0,3.785150,136.966568,mark
1,3.995588,135.359590,mark
2,4.206026,133.752611,mark
3,4.416463,132.145632,mark
4,4.626901,130.538653,mark


## Step B: Synchronize shot count with GNSS time
The step uses the RADAR 'x-values' (shot count) and a waypoint file's respective timestamps, or hard coded GNSS times, to synchronize the RADAR data with the track file. The function uses linear regression to correlate these two parameters, which therefore accounts for possible inaccuracies in the **sync_file** due to human error. If this relation gives a Pearson correlation (if e.g. a faulty point exists in the sync file) lower than 0.9, a warning will be given. Having a good correlation is integral to georeferencing the data.

The step also changes the 'typ' column to three separate columns representing return time (or lack thereof) of each respective type. The data are then resampled to 1s intervals.

In [5]:
def b_shot_sync(in_data, flight, evaluation=False):
    """Uses a 'X-value to waypoint/time stamp' synchronisation file,
    alternatively with a waypoint GPX-file, to synchronise shot count with the GNSS time."""
    
    # Read structure table
    waypoint_gpx = structure.loc[flight, "waypoints"]  # Name of waypoint gpx file, if needed
    sync_file = structure.loc[flight, "sync_file"]  # Synchronisation file
    wpt_sync = structure.loc[flight, "wpt_sync"]  # Boolean: If sync should be done with waypoints

    # If synchronisation should be done with waypoints
    if wpt_sync:
        gpx = gpxpy.parse(open(f"input/b_waypoints/{waypoint_gpx}"))  # Load waypoints gpx
        
        # Get waypoints (name as int) and respective time (in Unix time)
        wpts = pd.Series(
            [time.mktime(wpt.time.timetuple()) 
             for wpt in gpx.waypoints], [int(wpt.name) for wpt in gpx.waypoints])
        
        # Load csv with x values for respective waypoint
        wpt_x = pd.read_csv(f"input/b_shot_vs_wpt/{sync_file}", squeeze=True, index_col=0)  
        x_times = wpt_x.apply(lambda x: wpts[x])  # Use waypoint names (indices) to get respective time

    # If synchronisation should be done using time stamps
    else:
        x_times = pd.read_csv("input/b_shot_vs_wpt/" + sync_file, squeeze=True, index_col=0)
        
        # Seconds from DateTime string
        x_times = x_times.apply(
            lambda x: time.mktime(datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S").timetuple()))  

    # If evaluation is True, return a graph of x-values in relation to time (Unix-time)
    if evaluation:
        x_times.plot(marker="+")
        plt.title(flight)
        plt.show()
    
    # Check correlation between x-values and time, and warn if poor
    corr = np.corrcoef(x_times, x_times.index)[1, 0]
    if corr < 0.9:
        print(f"WARNING: {flight} sync file shows poor correlation (r={round(corr, 4)})")

    # Time where x=0; Regression between X-values and Unix-times.
    # Extract m-value (where x=0) and convert back to DateTime.
    x_0 = datetime.datetime.fromtimestamp(stats.linregress(x_times.index, list(x_times))[1])

    # Make new column "time" using the x_0 time added with shot number / 2 (shoots twice a second)
    in_data["time"] = in_data["x"].apply(lambda x: x_0 + datetime.timedelta(seconds=x / 2))
    in_data = in_data.set_index("time")[["y", "typ"]]  # Set time as index, and only keep y and typ
    

    # Resample the data in 1s intervals, and make columns with depth for each signal.
    types = pd.DataFrame(
        {typ: typ_df.resample("1S").mean().iloc[:, 0] for typ, typ_df in in_data.groupby("typ")})
    
    return types

**Data preview**

In [6]:
b_preview_data = b_shot_sync(a_preview_data, flight=preview_flight)
b_preview_data.dropna().head()

,fuffens,glacier,mark
time,,,
2012-04-16 09:18:57,146.004631,54.606126,200.920350
2012-04-16 09:18:58,144.795451,53.805664,209.217710
2012-04-16 09:18:59,143.586272,53.005203,217.389633
2012-04-16 09:19:00,142.377093,52.204742,225.561556
2012-04-16 09:19:01,141.908656,51.404280,232.990576


## Step C: Georeference the RADAR data using a GNSS track
With the correct time of the shots being known in the previous step, the GNSS and RADAR data can be joined using the time index. The GNSS track has to have the same frequency however, so it is consequently resampled to 1s intervals, using mean upsampling or linear interpolation for downsampling, or if gaps exist in the data. The GPX track input WGS 1984 coordinates are also projected into the specified projected system (epsg:3006 = SWEREF 99TM).

In [7]:
def c_georeference(in_data, flight):
    """Uses a 'track GPX' file to georeference the synchronised input data"""
    
    gpx_name = structure.loc[flight, "tracks"]
    
    if gpx_name.endswith(".gpx"):  # Load gpx file as DataFrame
        gpx = gpxpy.parse(open("input/c_tracks/" + gpx_name))

        # Extract data for DataFrame
        columns = ['Longitude', 'Latitude', 'Altitude']
        gpx_df = pd.DataFrame(columns=columns)
        for track in gpx.tracks:
            for segment in track.segments:
                for i, point in enumerate(segment.points):
                    # Append to gpx_df
                    gpx_df.loc[point.time, columns] = point.longitude, point.latitude, point.elevation  
    
    elif gpx_name.endswith(".p"):  # Exception made for 'anders1' flight. Loads prepared pickled dataframe
        gpx_df = pickle.load(open("input/c_tracks/" + gpx_name, "rb"))


    # Project geographic coordinates to SWEREF99TM
    in_proj = pyproj.Proj(init="epsg:4326")  # WGS1984
    out_proj = pyproj.Proj(init="epsg:3006") # SWEREF99TM
    
    for i, row in gpx_df.iterrows():
        x, y = pyproj.transform(in_proj, out_proj, row.Longitude, row.Latitude)
        gpx_df.loc[i, "Easting"] = x
        gpx_df.loc[i, "Northing"] = y
    
    # Mean downsampling (if freq < 1s), and interpolated gpx (if gpx is sampled over >1s)
    gpx_i = gpx_df.resample("1S").agg(lambda x: np.mean(x, axis=0)).interpolate()  
   
    joined = in_data.join(gpx_i)  # Join gpx data with radar data 
    
    return joined[["Easting", "Northing", "Altitude", "mark", "glacier", "fuffens"]]

**Data preview**

In [8]:
c_preview_data = c_georeference(b_preview_data, flight=preview_flight)
c_preview_data.dropna().head()

,Easting,Northing,Altitude,mark,glacier,fuffens
time,,,,,,
2012-04-16 09:18:57,654603.710526,7.556587e+06,1315.20,200.920350,54.606126,146.004631
2012-04-16 09:18:58,654579.098396,7.556585e+06,1319.55,209.217710,53.805664,144.795451
2012-04-16 09:18:59,654554.486266,7.556582e+06,1323.90,217.389633,53.005203,143.586272
2012-04-16 09:19:00,654530.237682,7.556580e+06,1327.50,225.561556,52.204742,142.377093
2012-04-16 09:19:01,654505.989098,7.556577e+06,1331.10,232.990576,51.404280,141.908656


## Step D: Shot return time to altitude conversion
The step takes the return time values and converts them into reflection distance, and subsequently to elevation using the GNSS altitudes. Since the signal travels in a vastly different speed through air vs. in ice, this is accounted for using two different speeds for each respective setting.

The distance to the glacier surface $d_g$ (if there is any at that specific shot) is first measured using air speed $v_a$:

$$ d_g = t_g \times v_a \times \frac{1}{512}$$

The englacial signal distance $d_e$, if relevant, is then defined using the following formula:

$$ d_e = d_g + v_e (t_e - t_g) \times \frac{1}{512}$$

Where $v_e$ is the englacial signal velocity and $t_e$ is the englacial signal time. All distances are divided by 512 to get the distance in meters.
Glacier travel time can also be expressed through glacier distance and air velocity, which is easier in the algorithm:

$$t_g = \frac{d_g \times 512}{v_a} $$

In [9]:
def d_depth_from_y(in_data, flight):
    """Takes return time (y) values and converts them to GNSS derived altitude"""

    # RADAR signal velocities
    v0 = 300  # In air
    v1 = 170  # In ice

    def depth_ice(time, glacier_dist):  # In cases with glacier penetration (including air travel time)
        t_g = (glacier_dist * 512) / v0
        return glacier_dist + (v1 * (time - t_g)) / 512

    def depth_air(val):  # In cases without glacier penetration.
        return (val / 512) * v0

    # Split dataset to data with or without glacier penetration
    glacier_data = in_data[in_data["glacier"].notnull()].copy()
    mark_data =    in_data[in_data["glacier"].isnull()].copy()
    
    # Remove any fuffens if it doesn't have a glacier value (needed for depth calculation)
    mark_data.loc[:, "fuffens"] = np.NaN
    
    # Calculate depth for values featuring no glacier penetration (only through air)
    glacier_data["glacier"] = depth_air(glacier_data["glacier"])
    mark_data["mark"]    =    depth_air(mark_data["mark"])
    
    # Calculate depth for values featuring glacier penetration    
    for col in ["mark", "fuffens"]:
        glacier_data[col] = depth_ice(glacier_data[col], glacier_data["glacier"])

    # Concatenate the two split DataFrames
    data = pd.concat([glacier_data, mark_data]).sort_index()

    # Convert depths to altitudes by suptracting GNSS altitude with depth
    data[["mark", "fuffens", "glacier"]] = data[["mark", "fuffens", "glacier"]].apply(
        lambda x: data["Altitude"] - x)

    return data

**Data preview**

In [10]:
d_preview_data = d_depth_from_y(c_preview_data, flight=preview_flight)
d_preview_data.dropna().head()

,Easting,Northing,Altitude,mark,glacier,fuffens
time,,,,,,
2012-04-16 09:18:57,654603.710526,7.556587e+06,1315.20,1234.623328,1283.204223,1252.857063
2012-04-16 09:18:58,654579.098396,7.556585e+06,1319.55,1236.421588,1288.023243,1257.811791
2012-04-16 09:18:59,654554.486266,7.556582e+06,1323.90,1238.261496,1292.842264,1262.766518
2012-04-16 09:19:00,654530.237682,7.556580e+06,1327.50,1239.351404,1296.911284,1266.971246
2012-04-16 09:19:01,654505.989098,7.556577e+06,1331.10,1240.687980,1300.980304,1270.930024


## Step E: Altitude correction using DEM
The GNSS altitude measurements considered to be unreliable, and the altitudes of every point featuring a glacier surface is corrected using a DEM. The average offset is also applied onto data points with no glacier surface. The ground is not used, due to many erroneous ground points seemingly existing, maybe due to the helicopter flying close to mountain sides.

A possible error in this might be that every point is corrected using a single DEM, and if large melt has occurred within the study period, e.g. 2012-2017, there will be an equally large error if the englacial properties have not changed accordingly. A solution would be to use multiple DEM's, possibly with linear interpolation between the two. The errors might however not be large enough to make a difference, but they need to be considered.

In [11]:
def e_correct_alt(in_data, flight):

    glacier_data = in_data[in_data["glacier"].notnull()].copy()
    mark_data =    in_data[in_data["glacier"].isnull()].copy()

    dem = gr.SingleBandRaster("input/Marma17_DEM_031017.tif")

    differences = []
    for i, row in glacier_data.iterrows():
        # Difference between GNSS Altitude + Depth and DEM Elevation
        diff = dem.value_at_coords(row["Easting"], row["Northing"]) - row["glacier"]  
        differences.append(diff)

        glacier_data.loc[i, ["Altitude", "glacier", "mark", "fuffens"]] += diff

    # Apply mean offset to ground with no glacier present.
    mark_data[["Altitude", "mark"]] += np.mean(differences)

    data = pd.concat([glacier_data, mark_data]).sort_index()

    return data

**Data preview**

In [12]:
e_preview_data = e_correct_alt(d_preview_data, flight=preview_flight)
e_preview_data.dropna().head()

,Easting,Northing,Altitude,mark,glacier,fuffens
time,,,,,,
2012-04-16 09:18:57,654603.710526,7.556587e+06,1394.685596,1314.108925,1362.689819,1332.342660
2012-04-16 09:18:58,654579.098396,7.556585e+06,1397.876000,1314.747587,1366.349243,1336.137791
2012-04-16 09:18:59,654554.486266,7.556582e+06,1400.907956,1315.269452,1369.850220,1339.774474
2012-04-16 09:19:00,654530.237682,7.556580e+06,1404.080171,1315.931575,1373.491455,1343.551417
2012-04-16 09:19:01,654505.989098,7.556577e+06,1407.221136,1316.809116,1377.101440,1347.051160


## Main application loop
Here all of the above steps are executed. It takes a list of flight names (RADAR series names), alternatively a single name or "all" (every name in the structure file). By default it writes a csv for each flight, and returns a dictionary {name: DataFrame}. If the **unified** parameter is True, it writes a single csv with the flight names as a separate column, and returns a respective DataFrame.

The **tag** parameter sets a specific name for the output file (e.g. tag="newest" -> unified_newest.csv). The default value is the date as YYMMDD of execution.


In [ ]:
auto_tag = datetime.datetime.today().strftime("%Y%m%d")[2:]  # YYMMDD

def prepare_radar(flights, save=False, tag=auto_tag, unified=False):

    # If 'all' is entered, use every flight in the structure table indices
    if flights == "all":
        flights = list(structure.index.values)
    # If input is a single flight as a string, convert it to a list for compatibility
    elif isinstance(flights, str):
        flights = [flights]

    # Properties of the different steps
    steps = ["a", "b", "c", "d", "e"]
    functions = [a_read_radar, b_shot_sync, c_georeference, d_depth_from_y, e_correct_alt]
    step_function = dict(zip(steps, functions))

    ###################
    # Perform steps
    ###################
    data = {}
    flight_data = pd.DataFrame()  # Needed to not raise an error at A
    for flight in flights:
        for step in steps:
            flight_data = step_function[step](flight_data, flight)
        
        # Add flight data to data dictionary
        data[flight] = flight_data
        
        # Save data
        if save and not unified:
            flight_data.to_csv(f"export/{flight}_{tag}.csv")
        
        print(f"{flight} finished.")

    # Option to return a DataFrame with a flight column, instead of dictionary with flight as a key
    if unified:
        all_data = pd.DataFrame()
        for name, df in data.items():
            df["Name"] = name
            all_data = pd.concat([all_data, df])
        all_data.to_csv(f"export/unified_{tag}.csv")
        
        return all_data
    
    if len(flights) == 1:
        return flight_data
    else:
        return data

**Data preview**

In [ ]:
data = prepare_radar("all", save=True, unified=True)
print("Files in 'export/' folder:", os.listdir("export/")[1:])

erik1 finished.


## Point cloud preparation
This function prepares three csv's (one for each return type) and prepares them to be easily read by point cloud softwares, such as CloudCompare. Color is also given appropriately.

In [ ]:
def export_points(tag=auto_tag):
    data = pd.read_csv(f"export/unified_{tag}.csv", index_col=0)  # Reads unified csv with specified tag
    
    # Convert data names to integers (sorted alphabetically)
    names = data["Name"].unique().tolist()
    names.sort()
    data["Name"] = data["Name"].apply(lambda x: names.index(x))
    
    
    colors = [(190, 121, 74), (56, 57, 255), (248, 57, 4)]  # Brown, blue and red for respective data type
    
    # Create one file for each data type
    data.drop("Altitude", axis=1, inplace=True)
    for i, col in enumerate(["mark", "glacier", "fuffens"]):
        csv = data.reset_index(drop=True)
        
        # Add color columns
        for c_i, c in enumerate(["r", "g", "b"]):
            csv[c] = colors[i][c_i]
        
        # Write csv with correctly ordered columns
        csv = csv[["Easting", "Northing", col, "r", "g", "b", "Name"]].dropna()
        csv.to_csv(f"export/clouds/radar_{col}_{tag}.csv", index=False)

**Data Preview**

In [ ]:
export_points()
print("Files in 'export/clouds' folder:", os.listdir("export/clouds")[1:])